In [3]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image

In [4]:
gpu_id = 1  # Change this to the index of the desired GPU
torch.cuda.set_device(gpu_id)

In [5]:
DATADIR = "RediMinds/Data_Cleaned/"
categories = ["Anaesthesia_machine","baby_incubator","Bone_saws","C_arm","colonoscope","Curved_Mayo_Scissor","difibrillator","Electrocautery_devices","fetal_doppler","forceps","Heart_Lung_Machine","IABP","IMRT","infusion_pump","Laryngoscopes","mayfield_clamp","Needle_Biopsy_Device","phacoemulsification","Radiofrequency_Ablation_Device","Scalpel","Straight_Dissection_Clamp","Straight_Mayo_Scissor","Suction_Machine","ventilator","x_ray"]
input_size = (224, 224)

In [6]:
class CustomDataset(Dataset):
    def __init__(self, data_dir, categories, input_size, transform=None):
        self.data_dir = data_dir
        self.categories = categories
        self.input_size = input_size
        self.transform = transform
        self.data = []
        self.labels = []
        self.load_data()

    def load_data(self):
        for cat in self.categories:
            path = os.path.join(self.data_dir, cat)
            class_num = self.categories.index(cat)
            for img_name in os.listdir(path):
                try:
                    img_path = os.path.join(path, img_name)
                    img = Image.open(img_path).convert("RGB")
                    img = img.resize(self.input_size)
                    if self.transform:
                        img = self.transform(img)
                    self.data.append(img)
                    self.labels.append(class_num)
                except Exception as e:
                    pass

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img = self.data[idx]
        label = self.labels[idx]
        return img, label

In [7]:
transform = transforms.Compose([
    transforms.RandomResizedCrop(input_size, scale=(0.8, 1.0)),
    # transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast = 0.25, saturation = 0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.2, 0.2, 0.2))
])

dataset = CustomDataset(DATADIR, categories, input_size, transform=transform)

train_size = int(0.70 * len(dataset))
validation_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - validation_size
train_dataset, validation_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size,validation_size, test_size])

In [8]:
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class CustomVGG16(nn.Module):
    def __init__(self, num_classes, fine_tune_layers=5):
        super(CustomVGG16, self).__init__()
        self.base_model = models.vgg16(pretrained=True)

        for param in self.base_model.parameters():
            param.requires_grad = False

        num_features = self.base_model.classifier[6].in_features
        self.base_model.classifier[6] = nn.Sequential(
            nn.Conv2d(num_features, 128, kernel_size=1),
            nn.AdaptiveAvgPool2d(1))

        self.cnn1 = nn.Conv2d(512,1024,kernel_size=3, padding=1)
        self.dropout1 = nn.Dropout(0.3)
        self.cnn2 = nn.Conv2d(1024,256,kernel_size = 3, padding=1)
        self.dropout2 = nn.Dropout(0.2)
        self.cnn3 = nn.Conv2d(256,64, kernel_size = 1, padding = 1)
        self.dropout3 = nn.Dropout(0.1)
        self.cnnl = nn.Conv2d(64,num_classes, kernel_size=1)

        layers_to_train = [
            self.base_model.features[-fine_tune_layers:],
            self.base_model.classifier[6],
            self.cnn1,
            self.cnn2,
            self.cnn3,
            self.cnnl]
        
        if fine_tune_layers > 0:
            for layer in layers_to_train:
                for param in layer.parameters():
                    param.requires_grad = True

    def forward(self, x):
        x = self.base_model.features(x)
        x = self.cnn1(x)
        x = torch.relu(x)
        x = self.dropout1(x)
        x = self.cnn2(x)
        x = torch.relu(x)
        x = self.cnn3(x)
        x = torch.relu(x)
        x = self.dropout3(x)
        x = self.cnnl(x)
        x = torch.flatten(x,1)
        return x

In [16]:
class CustomVGG16(nn.Module):
    def __init__(self, num_classes, fine_tune_layers=2):
        super(CustomVGG16, self).__init__()
        vgg16 = models.vgg16_bn(pretrained=models.VGG16_BN_Weights.IMAGENET1K_V1)

        # Freeze all layers by default
        for param in vgg16.parameters():
            param.requires_grad = False

        # Make the last block trainable
        for layer in vgg16.features[-fine_tune_layers:]:
            for param in layer.parameters():
                param.requires_grad = True

        for param in vgg16.classifier.parameters():
            param.requires_grad = True

        self.vgg16 = vgg16
        
        # Create the custom head (additional dense layers)
        self.head = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096,2048),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(2048, num_classes)
        )
        # self.cnn1 = nn.Conv2d(2048, 2048, kernel_size = 3, padding = 1)
        # self.dropout = nn.Dropout(0.5)
        # self.cnn2 = nn.Conv2d(2048, 512, kernel_size = 3, padding = 1)
        # self.dropout1 = nn.Dropout(0.4)
        # self.cnn3 = nn.Conv2d(512,64, kernel_size = 3, padding = 1)
        # self.dropout2 = nn.Dropout(0.3)
        # self.cnnl = nn.Conv2d(64, num_classes, kernel_size=3,padding=1)

    def forward(self, x):
        x = self.vgg16.features(x)
        x = torch.flatten(x, 1)
        x = self.head(x)
        # x = x.view(x.size(0), -1,1,1)
        # x = self.cnn1(x)
        # x = torch.relu(x)
        # x = self.dropout(x)
        # x = self.cnn2(x)
        # x = torch.relu(x)
        # x = self.dropout(x)
        # x = self.cnn3(x)
        # x = torch.relu(x)
        # x = self.dropout2(x)
        # x = self.cnnl(x)
        x = torch.flatten(x,1)
        return x

In [17]:
num_classes = len(categories)
model = CustomVGG16(num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

/home/kailashv/anaconda3/envs/mobilenet/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/kailashv/anaconda3/envs/mobilenet/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [18]:
for name, param in model.named_parameters():
    print(f"{name}: requires_grad={param.requires_grad}")

vgg16.features.0.weight: requires_grad=False
vgg16.features.0.bias: requires_grad=False
vgg16.features.1.weight: requires_grad=False
vgg16.features.1.bias: requires_grad=False
vgg16.features.3.weight: requires_grad=False
vgg16.features.3.bias: requires_grad=False
vgg16.features.4.weight: requires_grad=False
vgg16.features.4.bias: requires_grad=False
vgg16.features.7.weight: requires_grad=False
vgg16.features.7.bias: requires_grad=False
vgg16.features.8.weight: requires_grad=False
vgg16.features.8.bias: requires_grad=False
vgg16.features.10.weight: requires_grad=False
vgg16.features.10.bias: requires_grad=False
vgg16.features.11.weight: requires_grad=False
vgg16.features.11.bias: requires_grad=False
vgg16.features.14.weight: requires_grad=False
vgg16.features.14.bias: requires_grad=False
vgg16.features.15.weight: requires_grad=False
vgg16.features.15.bias: requires_grad=False
vgg16.features.17.weight: requires_grad=False
vgg16.features.17.bias: requires_grad=False
vgg16.features.18.weig

In [19]:
train_acc_history = []
train_loss_history = []
val_acc_history = []
val_loss_history = []

num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    train_loss, train_corrects = 0.0, 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        labels = torch.squeeze(labels)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        train_corrects += torch.sum(preds == labels).item()

    train_loss = train_loss / len(train_dataset)
    train_acc = train_corrects / len(train_dataset)
    train_acc_history.append(train_acc)
    train_loss_history.append(train_loss)

    model.eval()
    val_loss, val_corrects = 0.0, 0.0

    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            val_corrects += torch.sum(preds == labels).item()

    val_loss = val_loss / len(validation_dataset)
    val_acc = val_corrects / len(validation_dataset)

    val_acc_history.append(val_acc)
    val_loss_history.append(val_loss)

    print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

Epoch 1/50 | Train Loss: 2.8617 | Train Acc: 0.1465 | Val Loss: 2.1636 | Val Acc: 0.2891
Epoch 2/50 | Train Loss: 1.7510 | Train Acc: 0.4417 | Val Loss: 1.3224 | Val Acc: 0.5908
Epoch 3/50 | Train Loss: 0.9485 | Train Acc: 0.7022 | Val Loss: 1.0683 | Val Acc: 0.6564
Epoch 4/50 | Train Loss: 0.4541 | Train Acc: 0.8559 | Val Loss: 0.9771 | Val Acc: 0.7137
Epoch 5/50 | Train Loss: 0.2161 | Train Acc: 0.9360 | Val Loss: 0.9603 | Val Acc: 0.7263
Epoch 6/50 | Train Loss: 0.0889 | Train Acc: 0.9758 | Val Loss: 1.0446 | Val Acc: 0.7304
Epoch 7/50 | Train Loss: 0.0577 | Train Acc: 0.9836 | Val Loss: 1.0246 | Val Acc: 0.7374
Epoch 8/50 | Train Loss: 0.0445 | Train Acc: 0.9874 | Val Loss: 1.1029 | Val Acc: 0.7263
Epoch 9/50 | Train Loss: 0.0256 | Train Acc: 0.9955 | Val Loss: 1.0248 | Val Acc: 0.7570
Epoch 10/50 | Train Loss: 0.0122 | Train Acc: 0.9988 | Val Loss: 1.0839 | Val Acc: 0.7570
Epoch 11/50 | Train Loss: 0.0200 | Train Acc: 0.9949 | Val Loss: 1.1592 | Val Acc: 0.7318
Epoch 12/50 | Train

KeyboardInterrupt: 

In [ ]:
# Plot training and validation accuracy
import matplotlib.pyplot as plt

plt.plot(range(1, num_epochs+1), train_acc_history, label='Train')
plt.plot(range(1, num_epochs+1), val_acc_history, label='Validation')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot training and validation loss
plt.plot(range(1, num_epochs+1), train_loss_history, label='Train')
plt.plot(range(1, num_epochs+1), val_loss_history, label='Validation')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
model.eval()
test_loss, test_corrects = 0.0, 0.0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        test_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        test_corrects += torch.sum(preds == labels).item()

test_loss = test_loss / len(test_dataset)
test_acc = test_corrects / len(test_dataset)

print(f"Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.4f}")

---